In [1]:
cd ..

/experiments_motion


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys
import os
sys.path.append('./Motion')

In [4]:
import torch
import numpy as np
import zarr
import pandas as pd
from torch.utils.data import DataLoader, ConcatDataset
import yaml
from tqdm import tqdm
from IPython.display import display, HTML

In [6]:
from motion import Motion
from motion import Quaternion
from amass.skeleton import AMASSSkeleton
from amass.amass_torch_dataset import AMASSTorchDataset
from misc.helper import add_static_nodes

In [7]:
from metrics import MeanAngleL2Error, MeanPerJointPositionError, QuaternionAngle

## Load AMASS Skeleton

In [8]:
# Load skeleton configuration
with open('./config/amass_skeleton.yaml', 'r') as stream:
    skeleton = AMASSSkeleton(**yaml.safe_load(stream))

## Load Evaluation Data

In [9]:
dataset_path = './data/processed/amass'
datasets = ['Transitionsmocap', 'SSMsynced']

In [10]:
datasets_list = []
for dataset in datasets:
    z_index = zarr.open(os.path.join(dataset_path, dataset, 'poses_index.zarr'), 'r')
    z_poses = zarr.open(os.path.join(dataset_path, dataset, 'poses.zarr'), 'r')
    datasets_list.append(AMASSTorchDataset(z_index, z_poses, history_length=40, prediction_horizon=20))
dataset_eval = ConcatDataset(datasets_list)

## Define Eval Metrics

In [11]:
mae_l2_metric = MeanAngleL2Error(ignore_root=True) # We ignore the root rotation in the world for the origin joint
mpjpe_metric = MeanPerJointPositionError()

In [12]:
t = [1, 3, 7, 11, 15, 19]

# Evaluation of Model on AMASS dataset

## Load the Model

In [13]:
MODEL_PATH = './output/amass/deterministic'
CHECKPOINT = 'model'
CHECKPOINT_PATH = os.path.join(MODEL_PATH, CHECKPOINT + '.pth.tar')

In [18]:
# Load model config
with open(os.path.join(MODEL_PATH, 'config.yaml'), 'r') as file:
    model_config = yaml.safe_load(file)

In [19]:
model = Motion(skeleton, T=skeleton.nodes_type_id_dynamic, **model_config)
model.load_state_dict(torch.load(CHECKPOINT_PATH, map_location=torch.device('cpu')))
model.eval()
print('Model Loaded')

Model Loaded


## Evaluate on 10000 samples

In [20]:
eval_idx = list(torch.linspace(0, len(dataset_eval)-1, 10000, dtype=torch.int).numpy())
eval_subset = torch.utils.data.Subset(dataset_eval, eval_idx)
data_loader_eval = DataLoader(eval_subset, shuffle=False, batch_size=256)

In [24]:
def sorted_modes(p_q):
    qs = p_q.component_distribution.mean
    probs = p_q.mixture_distribution.probs
    w = probs
    probs_sorted, si = w.sort(-1, descending=True)
    mode_probs.append(probs_sorted)
    si = si.repeat(1 ,qs.shape[1], qs.shape[2], 1)
    q_mode = qs.gather(dim=-2, index=si.unsqueeze(-1).repeat(1, 1, 1, 1, 4)).squeeze(-2)
    return q_mode

In [25]:
mae_l2_metric.reset()
mpjpe_metric.reset()
modes = []
mode_probs = []
mode_s = []
with torch.no_grad():
    for x, y in tqdm(data_loader_eval):

        # Remove static nodes
        x_dynamic = Quaternion.qfix_positive_(x[:, :, skeleton.dynamic_nodes])

        # Run Model 
        p_q, _, _, _ = model(x_dynamic, None, ph=20)

        # The mode of the Distribution are the rotations
        q_mode = p_q.mode
        # Add static rotations q = [1., 0., 0., 0.]
        q_mode_all = add_static_nodes(q_mode, skeleton.static_nodes, skeleton.dynamic_nodes)
        modes.append(add_static_nodes(sorted_modes(p_q).permute(0, 1, 3, 2, 4), skeleton.static_nodes, skeleton.dynamic_nodes))

        mae_l2_metric.update((q_mode_all, y))

        pos_mode_all = skeleton(q_mode_all.view(-1, 22, 4)).view(-1, 20, 22, 3)
        pos_y = skeleton(y.view(-1, 22, 4)).view(-1, 20, 22, 3)
        mpjpe_metric.update((pos_mode_all, pos_y))

  0%|                                                                                                                                                                                                                  | 0/40 [00:00<?, ?it/s]

  2%|█████                                                                                                                                                                                                     | 1/40 [00:03<02:25,  3.73s/it]

  5%|██████████                                                                                                                                                                                                | 2/40 [00:07<02:24,  3.81s/it]

  8%|███████████████▏                                                                                                                                                                                          | 3/40 [00:11<02:22,  3.84s/it]

 10%|████████████████████▏                                                                                                                                                                                     | 4/40 [00:15<02:18,  3.85s/it]

...

 88%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                         | 35/40 [02:11<00:19,  3.80s/it]

 90%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                    | 36/40 [02:15<00:15,  3.77s/it]

 92%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉               | 37/40 [02:19<00:11,  3.76s/it]

 95%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉          | 38/40 [02:23<00:07,  3.77s/it]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [02:27<00:00,  3.68s/it]

In [26]:
print(f"{mae_l2_metric.y.shape[0]} Sequences used for evaluation.")

10000 Sequences used for evaluation.


In [27]:
e1f = Quaternion.euler_angle_(mae_l2_metric.y_pred.contiguous(), 'zyx').flatten(start_dim=-2)
e2f = Quaternion.euler_angle_(mae_l2_metric.y.contiguous(), 'zyx').flatten(start_dim=-2)
diff_modes = torch.remainder(e1f - e2f + np.pi, 2 * np.pi) - np.pi

In [28]:
def mae_l2_modes(modes, y):
    e1f = Quaternion.euler_angle_(modes.contiguous(), 'zyx').flatten(start_dim=-2)
    e2f = Quaternion.euler_angle_(y.contiguous(), 'zyx').flatten(start_dim=-2)
    diff_modes = torch.remainder(e1f - e2f + np.pi, 2 * np.pi) - np.pi
    _, idx = diff_modes.norm(dim=-1).mean(1).min(-1)
    return diff_modes.norm(dim=-1).gather(dim=-1, 
                             index=idx.unsqueeze(-1).unsqueeze(-1).repeat(1, diff_modes.shape[1], 1)).mean(0).squeeze()

In [29]:
def mpjpe_modes(modes, y):
    pos_modes = skeleton(modes.view(-1, 22, 4)).view(modes.shape[:-1] + (3,))
    pos_y = skeleton(y.view(-1, 22, 4)).view(y.shape[:-1] + (3,))
    diff_modes = (pos_y - pos_modes).norm(dim=-1)
    _, idx = diff_modes.mean(dim=[1, -1]).min(-1)
    return diff_modes.mean(-1).gather(dim=-1, 
                             index=idx.unsqueeze(-1).unsqueeze(-1).repeat(1, diff_modes.shape[1], 1)).squeeze().mean(0)

In [30]:
mae_l2_besto3 = mae_l2_modes(torch.cat(modes, dim=0)[..., :3, 1:, :], mae_l2_metric.y[..., 1:, :].unsqueeze(2))
mae_l2_besto5 = mae_l2_modes(torch.cat(modes, dim=0)[..., :5, 1:, :], mae_l2_metric.y[..., 1:, :].unsqueeze(2))
mae_l2_besto10 = mae_l2_modes(torch.cat(modes, dim=0)[..., :10, 1:, :], mae_l2_metric.y[..., 1:, :].unsqueeze(2))

In [31]:
mpjpe_besto3 = mpjpe_modes(torch.cat(modes, dim=0)[..., :3, :, :].contiguous(), mae_l2_metric.y.unsqueeze(2))
mpjpe_besto5 = mpjpe_modes(torch.cat(modes, dim=0)[..., :5, :, :].contiguous(), mae_l2_metric.y.unsqueeze(2))
mpjpe_besto10 = mpjpe_modes(torch.cat(modes, dim=0)[..., :10, :, :].contiguous(), mae_l2_metric.y.unsqueeze(2))

In [32]:
mae_l2 = mae_l2_metric.compute().numpy()
mpjpe = mpjpe_metric.compute().numpy()

In [33]:
result = pd.DataFrame()
result.insert(0, 'Mean Angle Error (L2) BO10', pd.Series(data=mae_l2_besto10.numpy(), index=0.05*np.arange(1, mae_l2.shape[0] + 1)))
result.insert(0, 'Mean Angle Error (L2) BO5', pd.Series(data=mae_l2_besto5.numpy(), index=0.05*np.arange(1, mae_l2.shape[0] + 1)))
result.insert(0, 'Mean Angle Error (L2) BO3', pd.Series(data=mae_l2_besto3.numpy(), index=0.05*np.arange(1, mae_l2.shape[0] + 1)))
result.insert(0, 'Mean Angle Error (L2)', pd.Series(data=mae_l2, index=0.05*np.arange(1, mae_l2.shape[0] + 1)))
result.insert(0, 'Mean per Joint Position Error BO10', pd.Series(data=mpjpe_besto10*1000, index=0.05*np.arange(1, mpjpe.shape[0] + 1)))
result.insert(0, 'Mean per Joint Position Error BO5', pd.Series(data=mpjpe_besto5*1000, index=0.05*np.arange(1, mpjpe.shape[0] + 1)))
result.insert(0, 'Mean per Joint Position Error BO3', pd.Series(data=mpjpe_besto3*1000, index=0.05*np.arange(1, mpjpe.shape[0] + 1)))
result.insert(0, 'Mean per Joint Position Error', pd.Series(data=mpjpe*1000, index=0.05*np.arange(1, mpjpe.shape[0] + 1)))
result.index.set_names(['T'], inplace=True)

In [34]:
display(HTML(result.iloc[t].to_html(float_format=lambda x: '%.2f' % x)))

,Mean per Joint Position Error,Mean per Joint Position Error BO3,Mean per Joint Position Error BO5,Mean per Joint Position Error BO10,Mean Angle Error (L2),Mean Angle Error (L2) BO3,Mean Angle Error (L2) BO5,Mean Angle Error (L2) BO10
T,,,,,,,,
0.1,19.14,19.02,19.12,19.12,0.42,0.42,0.42,0.42
0.2,38.19,37.33,37.37,37.37,0.76,0.74,0.74,0.74
0.4,64.14,59.91,58.76,58.76,1.08,1.01,0.99,0.99
0.6,76.92,69.99,67.52,67.52,1.22,1.12,1.09,1.09
0.8,84.25,76.56,73.82,73.82,1.31,1.20,1.17,1.17
1.0,89.90,82.83,80.95,80.95,1.38,1.28,1.27,1.27


In [40]:
np.save('probs',np.concatenate(mode_probs, axis=0))

# Evaluation of Weighted Mean on AMASS dataset

## Evaluate on 10000 samples

In [35]:
eval_idx = list(torch.linspace(0, len(dataset_eval)-1, 10000, dtype=torch.int).numpy())
eval_subset = torch.utils.data.Subset(dataset_eval, eval_idx)
data_loader_eval = DataLoader(eval_subset, shuffle=False, batch_size=256)

In [36]:
mae_l2_metric.reset()
mpjpe_metric.reset()
with torch.no_grad():
    for x, y in tqdm(data_loader_eval):

        # Remove static nodes
        x_dynamic = Quaternion.qfix_positive_(x[:, :, skeleton.dynamic_nodes])

        # Run Model 
        p_q, _, _, _ = model(x_dynamic, None, ph=20)

        
        # The mode of the Distribution are the rotations
        q_mode = p_q.weighted_mean
        # Add static rotations q = [1., 0., 0., 0.]
        q_mode_all = add_static_nodes(q_mode, skeleton.static_nodes, skeleton.dynamic_nodes)
        
        mae_l2_metric.update((q_mode_all, y))

        pos_mode_all = skeleton(q_mode_all.view(-1, 22, 4)).view(-1, 20, 22, 3)
        pos_y = skeleton(y.view(-1, 22, 4)).view(-1, 20, 22, 3)
        mpjpe_metric.update((pos_mode_all, pos_y))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [02:22<00:00,  3.55s/it]


In [37]:
print(f"{mae_l2_metric.y.shape[0]} Sequences used for evaluation.")

10000 Sequences used for evaluation.


In [38]:
skeleton(q_mode_all.view(-1, 22, 4))[1:].mean()

tensor(-0.0241)

In [39]:
mae_l2 = mae_l2_metric.compute().numpy()
mpjpe = mpjpe_metric.compute().numpy()

In [40]:
result = pd.DataFrame()
result.insert(0, 'Mean Angle Error (L2)', pd.Series(data=mae_l2, index=0.05*np.arange(1, mae_l2.shape[0] + 1)))
result.insert(0, 'Mean per Joint Position Error', pd.Series(data=mpjpe*1000, index=0.05*np.arange(1, mpjpe.shape[0] + 1)))
result.index.set_names(['T'], inplace=True)

In [41]:
display(HTML(result.iloc[t].to_html(float_format=lambda x: '%.2f' % x)))

,Mean per Joint Position Error,Mean Angle Error (L2)
T,,
0.1,19.05,0.42
0.2,37.81,0.76
0.4,63.00,1.05
0.6,75.28,1.19
0.8,82.31,1.27
1.0,87.48,1.33


In [42]:
display(HTML(result.iloc[t].to_html(float_format=lambda x: '%.2f' % x)))

,Mean per Joint Position Error,Mean Angle Error (L2)
T,,
0.1,19.05,0.42
0.2,37.81,0.76
0.4,63.00,1.05
0.6,75.28,1.19
0.8,82.31,1.27
1.0,87.48,1.33
